In [1]:
import os
import pandas as pd
import numpy as np

In [7]:
model = pd.read_excel("Food_ImpactFactors(ONmodel)_updated_categories.xlsx")
on_model = list(model['food_items'])
#on_model

In [12]:
import pandas as pd

file_path = "export_result_oct21.xlsx"

months = [
    "Dec23","Jan24","Feb24","Mar24","Apr24","May24",
    "Jun24","Jul24","Aug24","Sep24","Oct24","Nov24"
]

xl = pd.ExcelFile(file_path)

for sheet in months:
    df = pd.read_excel(xl, sheet_name=sheet)

    # === TABLE 1 === (A–E → columns 0:5, dynamic length)
    # Detect where table 2 starts ("food_items" in col A)
    food_start = df.index[df.iloc[:, 0].astype(str).str.lower().eq("food_items")]
    if len(food_start) > 0:
        end_t1 = food_start[0]
    else:
        end_t1 = len(df)
    table1 = df.iloc[:end_t1, 0:5].dropna(how="all")

    # === TABLE 2 === (A–D → columns 0:4, starts at "food_items")
    if len(food_start) > 0:
        start_t2 = food_start[0]
        # find next blank row to determine end
        after = df.iloc[start_t2+1:, 0].isna()
        if after.any():
            end_t2 = after.idxmax()
        else:
            end_t2 = len(df)
        table2 = df.iloc[start_t2:end_t2, 0:4].dropna(how="all")
        table2.drop(table2.index[0], inplace=True)
    else:
        table2 = pd.DataFrame()
        table2.drop(table2.index[0], inplace=True)
        
    if table2.shape[1] == 4:
        table2.columns = ['food_items', 'quantity', 'sum_of_portionunitstandard_g', 'sum_of_portionunitstandard_kg']
       
    
    # === ANIMAL & SUMMARY (H–L: cols 7–12) ===
    animal = df.iloc[0:17, 7:12].dropna(how="all")
    animal_name = ['food_items', 'quantity', 'sum_of_portionunitstandard_g',
       'gwp_of_1_kg_food', 'gwp_x_kg']
    animal.columns = animal_name
    
    summary = df.iloc[18:23, 7:12].dropna(how="all")
    summary_name = ['variables', 'mass_kg', 'percentage_kg', 'gwp_total', 'percentage_gwp']
    summary.columns = summary_name

    # === ANIMAL_S & SUMMARY_S (O–S: cols 14–19) ===
    animal_s = df.iloc[0:17, 14:19].dropna(how="all")
    animal_s.columns = animal_name
    summary_s = df.iloc[18:23, 14:19].dropna(how="all")
    summary_s.columns = summary_name

    # === DROP completely empty columns (in case of blanks)
    table1 = table1.dropna(axis=1, how="all")
    table2 = table2.dropna(axis=1, how="all")
    if table2.shape[1] == 3:
        table2.columns = ['food_items', 'sum_of_portionunitstandard_g', 'sum_of_portionunitstandard_kg']
        #print(sheet)

    # === Assign to variables dynamically ===
    suffix = sheet[:3].lower()  # e.g., dec, jan, feb
    globals()[f"{suffix}_table1"]   = table1
    globals()[f"{suffix}_table2"]   = table2
    globals()[f"{suffix}_animal"]   = animal
    globals()[f"{suffix}_summary"]  = summary
    globals()[f"{suffix}_animal_s"] = animal_s
    globals()[f"{suffix}_summary_s"]= summary_s

    #print(f"✅ Extracted all tables for {sheet}")

print("All months processed and loaded into variables.")

All months processed and loaded into variables.


In [31]:
dec_table1

,food_items,quantity,sum_of_portionunitstandard_g,gwp_of_1_kg_food,gwp_x_kg
0,A_beef_consumed_ON,16098,1444621.5,55.64676,80388.505901
1,Vegetables_MontegoBlend,8502,638964,3.44,2198.036160
2,A_mixed_meat_consumed_ON,701,43586,33.62227,1465.460260
3,A_butter_consumed_ON,17511,59515.5,19.79185,1177.921849
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,721,53314,15.17,808.773380
...,...,...,...,...,...
73,Supp_Isosource+HN_MilkProtein_SoyProtein_Water...,614,145190,13.33,1935.382700
74,B_tea_brewed_consumed_ON,39279,9774750,0.03785,369.974288
75,Greenbeans_Proxy_chickpeas,5288,307206,0.92,282.629520
76,FrenchToast_Proxy_Eggs_Bread,6164,184956.2,6.12,1131.931944


In [35]:
dec_table1['food_items'].isin(['A_beef_consumed_ON'])

0      True
1     False
2     False
3     False
4     False
      ...  
73    False
74    False
75    False
76    False
77    False
Name: food_items, Length: 78, dtype: bool

In [55]:
def check(ch):
    months = ['dec',
             'jan',
             'feb',
             'mar',
             'apr',
             'may',
             'jun',
             'jul',
             'aug',
             'sep',
             'oct',
             'nov']
    
    have = []
    
    for name in months:
        if globals()[f"{name}_table1"]['food_items'].isin([ch]):
            have.append(name)
        
    print('have:', have)
    #if months==have:
    #    print('all in')

In [30]:
for name in months:
    name = name[:3].lower()
    #table1 = globals()[f"{name}_table1"]
    #print(table1)
    print(name)
    print(globals()[f"{name}_table1"].head())
    print("\n")

dec
                               food_items quantity  \
0                      A_beef_consumed_ON    16098   
1                 Vegetables_MontegoBlend     8502   
2                A_mixed_meat_consumed_ON      701   
3                    A_butter_consumed_ON    17511   
4  Pizza_Flatbread_Cheese_Flatbred_Tomato      721   

  sum_of_portionunitstandard_g gwp_of_1_kg_food      gwp_x_kg  
0                    1444621.5         55.64676  80388.505901  
1                       638964             3.44   2198.036160  
2                        43586         33.62227   1465.460260  
3                      59515.5         19.79185   1177.921849  
4                        53314            15.17    808.773380  


jan
                               food_items quantity  \
0                      A_beef_consumed_ON    17178   
1                 Vegetables_MontegoBlend     7986   
2                A_mixed_meat_consumed_ON      820   
3                    A_butter_consumed_ON    19011   
4  Pizza_Fl